# Chapter 17: Representation Learning and Generative Learning Using Autoencoders and GANs

Bab ini memperkenalkan dua arsitektur jaringan saraf yang kuat dan dapat belajar tanpa data berlabel (*unsupervised learning*): **Autoencoders** dan **Generative Adversarial Networks (GANs)**.

Keduanya mampu mempelajari representasi data yang padat (disebut *latent representations* atau *codings*), dan keduanya dapat digunakan sebagai model generatif untuk menghasilkan data baru yang mirip dengan data pelatihan.

* **Autoencoders** belajar untuk merekonstruksi inputnya kembali. Dengan memberikan batasan tertentu pada jaringan (misalnya, lapisan tengah yang lebih kecil), kita memaksanya untuk belajar representasi data yang efisien.
* **GANs** terdiri dari dua jaringan yang bersaing: *generator* yang mencoba membuat data palsu yang realistis, dan *discriminator* yang mencoba membedakan data asli dari data palsu. Kompetisi ini mendorong kedua jaringan untuk menjadi lebih baik.

## Autoencoders

**Autoencoder** adalah jaringan saraf yang terdiri dari dua bagian:
1. **Encoder:** Mengubah input menjadi representasi laten (coding).
2. **Decoder:** Mengubah representasi laten kembali menjadi output yang semirip mungkin dengan input asli.

Dengan memaksa representasi laten memiliki dimensi yang lebih kecil dari input (*undercomplete autoencoder*), kita memaksa jaringan untuk mempelajari fitur-fitur data yang paling penting.

### Stacked Autoencoders

Autoencoder dapat memiliki beberapa *hidden layer*, yang disebut **stacked autoencoders**. Ini membantunya mempelajari *coding* yang lebih kompleks. Arsitekturnya biasanya simetris.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Memuat dataset Fashion MNIST
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train_full[5000:].astype(np.float32) / 255
X_valid = X_train_full[:5000].astype(np.float32) / 255

# Membangun Stacked Autoencoder
stacked_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(30, activation="selu"),
])
stacked_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[30]),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])
stacked_ae = keras.models.Sequential([stacked_encoder, stacked_decoder])

stacked_ae.compile(loss="binary_crossentropy", optimizer=keras.optimizers.SGD(lr=1.5))
history = stacked_ae.fit(X_train, X_train, epochs=20, validation_data=(X_valid, X_valid))

### Visualisasi Rekonstruksi
Salah satu cara untuk mengevaluasi autoencoder adalah dengan membandingkan input asli dengan hasil rekonstruksinya.

In [ ]:
def plot_image(image):
    plt.imshow(image, cmap="binary")
    plt.axis("off")

def show_reconstructions(model, n_images=5):
    reconstructions = model.predict(X_valid[:n_images])
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plot_image(X_valid[image_index])
        plt.title("Original")
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plot_image(reconstructions[image_index])
        plt.title("Reconstructed")
    plt.show()

show_reconstructions(stacked_ae)

### Variational Autoencoders (VAEs)
VAEs adalah autoencoder *probabilistic* dan *generative*. Alih-alih menghasilkan *coding* yang pasti, *encoder* menghasilkan *mean* (μ) dan *standard deviation* (σ). *Coding* yang sebenarnya kemudian di-sampel secara acak dari distribusi Gaussian dengan parameter μ dan σ. Ini memungkinkan VAE untuk menghasilkan *instance* baru yang realistis.

## Generative Adversarial Networks (GANs)

GANs menggunakan pendekatan yang berbeda secara fundamental. Mereka terdiri dari dua jaringan yang bersaing:

* **Generator:** Mencoba menghasilkan data palsu yang terlihat nyata.
* **Discriminator:** Mencoba membedakan antara data nyata dari *training set* dan data palsu dari *generator*.

Selama pelatihan, kedua jaringan ini saling mendorong untuk menjadi lebih baik dalam permainan "kucing dan tikus" ini. Tujuan akhirnya adalah agar *generator* menjadi sangat baik dalam menghasilkan data sehingga *discriminator* tidak bisa lagi membedakannya dari data asli.

### Deep Convolutional GANs (DCGANs)

DCGANs adalah varian GAN yang menggunakan *convolutional layer* untuk menghasilkan gambar, yang terbukti sangat efektif. Ada beberapa panduan arsitektur kunci untuk membuat pelatihan DCGAN lebih stabil, seperti menggunakan *transposed convolution* di *generator* dan *strided convolution* di *discriminator*.

In [ ]:
# Contoh konseptual membangun DCGAN sederhana
codings_size = 100

generator = keras.models.Sequential([
    keras.layers.Dense(7 * 7 * 128, input_shape=[codings_size]),
    keras.layers.Reshape([7, 7, 128]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="same", activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2, padding="same", activation="tanh"),
])

discriminator = keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size=5, strides=2, padding="same", activation=keras.layers.LeakyReLU(0.2), input_shape=[28, 28, 1]),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(128, kernel_size=5, strides=2, padding="same", activation=keras.layers.LeakyReLU(0.2)),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation="sigmoid")
])

gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

### Visualisasi Gambar yang Dihasilkan GAN
Setelah melatih GAN, kita dapat menggunakannya untuk menghasilkan gambar-gambar baru yang belum pernah ada sebelumnya.

In [ ]:
# Fungsi untuk menghasilkan dan memplot gambar
def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    plt.figure(figsize=(n_cols * 1.5, n_rows * 1.5))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

# Menghasilkan beberapa gambar (setelah model dilatih)
noise = tf.random.normal(shape=[10, codings_size])
generated_images = generator.predict(noise)

plot_multiple_images(generated_images, n_cols=5)